In [2]:
CURRENT_DIR = '../'
import json
import os
import pickle
from glob import glob
from tqdm.notebook import tqdm

with open(os.path.join(CURRENT_DIR, 'IEMOCAP/labels.json'), 'r') as stream:
    labels_original_all = json.load(stream)
    
    
declare = {}
for DATASET in ['train', 'val', 'test']:
    declare[DATASET] = {}

    with open(os.path.join(
        CURRENT_DIR, f'scripts/iemocap_pkl-DeCLaRe/{DATASET}/sentences.pkl'), 'rb') as stream: 
        declare[DATASET]['sentences'] = pickle.load(stream)

    with open(os.path.join(
        CURRENT_DIR, f'scripts/iemocap_pkl-DeCLaRe/{DATASET}/conversation_length.pkl'), 'rb') as stream: 
        declare[DATASET]['conversation_length'] = pickle.load(stream)

    with open(os.path.join(
        CURRENT_DIR, f'scripts/iemocap_pkl-DeCLaRe/{DATASET}/labels.pkl'), 'rb') as stream: 
        declare[DATASET]['labels'] = pickle.load(stream)
    
def read_orignal_utt(DATASET, uttid):
    with open(os.path.join(CURRENT_DIR, 'IEMOCAP', 'raw-texts', DATASET, uttid + '.json'), 'r') as stream:
        utt = json.load(stream)['Utterance']
    return utt

def get_easy_tokens(utt):
    assert isinstance(utt, str)
    
    utt = [token.lower() 
           for token in utt.split(' ') 
           if token not 
           in ['.', ' .', '. ', ' . ', '!', '?', '`','-', '_', '/', '"', "''"]]
    return utt

def count_same_words(utt1, utt2):
    count=0
    utt1 = get_easy_tokens(utt1)
    utt2 = get_easy_tokens(utt2)
#     print(utt1)
#     print(utt2)
    for token in utt1:
        if token in utt2:
            count+=1
    return count

def calc_utt_match(utt1, utt2):
    count1 = count_same_words(utt1, utt2)
    count2 = count_same_words(utt2, utt1)
    
    return (count1 / len(utt1)) * (count2 / len(utt2))


original_utts = {'train': {}, 'val': {}, 'test': {}}
for utt_path in glob(os.path.join(CURRENT_DIR, 'IEMOCAP/raw-texts/*/*.json')):
    DATASET = utt_path.split('/')[-2]
    uttid = os.path.basename(utt_path).split('.json')[0]
    
    with open(utt_path, 'r') as stream:
        original_utts[DATASET][uttid] = json.load(stream)['Utterance']
        
for DATASET in ['train', 'val', 'test']:
    for i, utts in enumerate(declare[DATASET]['sentences']):
        for j, utt in enumerate(utts):
            utt = ' '.join([token for token in utt if token not in ['<eos>', '<pad>']])
            utts[j] = utt
        assert len(utts) == len(declare[DATASET]['labels'][i])

utts_matched = {}        
for DATASET in tqdm(['train', 'val', 'test']):
    utts_matched[DATASET] = []
    assert len(declare[DATASET]['sentences']) == len(declare[DATASET]['labels'])
    for utts_declare, labels_declare in tqdm(zip(declare[DATASET]['sentences'], declare[DATASET]['labels'])):
        assert len(utts_declare) == len(labels_declare)
        for utt_declare, label_declare in tqdm(zip(utts_declare, labels_declare)):
            foo = [({'uttid': uttid, 
                     'utt_declare': utt_declare, 
                     'utt_original':utt_original,
                    'emotion_declare': label_declare,
                    'emotion_original': labels_original_all[DATASET][uttid]}, 
                    calc_utt_match(utt_declare, utt_original)) 
                   for uttid, utt_original in original_utts[DATASET].items()] 
            foo = sorted(foo, key=lambda x: -x[-1])
            
            utts_matched[DATASET].append(foo[0])


In [3]:
import random

In [24]:
random.choice(utts_matched['train'])

({'uttid': 'Ses02M_script02_2_M022',
  'utt_declare': 'you want to go get some champagne ?',
  'utt_original': 'You want to go get some champagne?',
  'emotion_declare': 2,
  'emotion_original': 'neutral'},
 0.030252100840336138)